In [2]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)
from pyspark.sql.functions import *
import json
import sys
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


from torchtext import data
from torchtext import datasets
import pickle
import spacy

from model import RNN, preprocess_tweet
nlp = spacy.load('en')

spark = SparkSession.builder.master('spark://spark-master:7077').config('spark.cores.max','2').getOrCreate()
sc = spark.sparkContext

In [3]:
trumpDF = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "Trump")\
.load()


bidenDF = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "Biden")\
.load()

In [4]:
with open('./model/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)


INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = 0

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

model.load_state_dict(torch.load('./model/tut2-model.pt', map_location=torch.device('cpu')))

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


<All keys matched successfully>

In [5]:
def predict_sentiment(sentence):
    sentence = preprocess_tweet(sentence)
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    if prediction.item() >= 0.5:
        return 'Positive'
    else:
        return 'Negative'

In [6]:
schema = StructType([   
        StructField("time", StringType(), True),
        StructField("text", StringType(), True),
        StructField("retweet_count", DoubleType(), True),
        StructField("location", StringType(), True),
        StructField("favorite_count", DoubleType(), True),
        StructField("user_id", StringType(), True),
        StructField("place", StringType(), True),
        StructField("user_followers_count", StringType(), True),
    
])


def castData(schema, df):
    df = df.selectExpr("CAST(value AS STRING)")
    df = df.select(from_json(col("value"), schema).alias("data")).select("data.*")
    one_row_udf = udf(predict_sentiment, StringType())
    df = df.withColumn('sentiment', one_row_udf(col('text')))
    
    return df
    
trumpDF = castData(schema, trumpDF)
bidenDF = castData(schema, bidenDF)


In [7]:
query = trumpDF.writeStream.queryName("device_counts").format("memory")\
    .start()

In [12]:
spark.sql('SELECT count(*) FROM device_counts').show(20, False)

+--------+
|count(1)|
+--------+
|0       |
+--------+



In [9]:
trumpDF.writeStream.trigger(processingTime='5 seconds').queryName("trump_tweets")\
.format("csv").outputMode("append").option("checkpointLocation", "hdfs://namenode:9000/checkpoints_Trump")\
.option('path', 'hdfs://namenode:9000/data/trump.csv').start()


In [10]:
bidenDF.writeStream.trigger(processingTime='5 seconds').queryName("biden_tweets")\
.format("csv").outputMode("append").option("checkpointLocation", "hdfs://namenode:9000/checkpoints_Biden")\
.option('path', 'hdfs://namenode:9000/data/biden.csv').start()

In [ ]:
import sys
sys.version.split(' ')[0]